# The fast spectral method

In [3]:
%matplotlib inline
import numpy as np
from math import pi
from scipy import special
import matplotlib.pyplot as plt
import pyfftw
import cupy as cp

# import os
# os.chdir('../../../src')

from pykinetic.collision.fast_spec_col_2d import FastSpectralCollision2D
from utility import get_config, plot_2d

In [2]:
def bkw_f(v):
    t = 0.5
    K = 1 - 0.5*np.exp(-t/8)
    v_norm = (v**2)[:,None] + v**2
    return 1/(2*pi*K**2)*np.exp(-0.5*v_norm/K)*(2*K-1+0.5*v_norm*(1-K)/K)

def ext_Q(v):
    t = 0.5
    K = 1 - np.exp(-t/8)/2
    dK = np.exp(-t/8)/16
    v_norm = (v**2)[:,None] + v**2
    df = (-2/K+v_norm/(2*K**2))*bkw_f(v) + 1/(2*pi*K**2)*np.exp(-v_norm/(2*K))*(2-v_norm/(2*K**2))
    return df*dK

In [3]:
config = get_config('./configs/config_0d_2d.json')
config.e = 1
Q = FastSpectralCollision2D(config)

In [4]:
getattr(Q, 'e')

In [5]:
f0 = bkw_f(Q.v)
plot_2d(f0, Q.v, 0)

In [20]:
%time Q1 = Q.col_new(bkw_f(Q.v))
%time Q2 = Q.col_sep(bkw_f(Q.v))
%time Q3 = Q.col_sep_gpu_copy(bkw_f(Q.v))
np.max(np.abs(Q1 - ext_Q(Q.v))), np.max(np.abs(Q2 - ext_Q(Q.v))), np.max(np.abs(Q3 - ext_Q(Q.v)))

In [11]:
from configparser import ConfigParser
from collisionV2.spectral_mesh import SpectralMesh
from collisionV2.inelastic_collisions import FSInelasticVHSCollision

cfg = ConfigParser()
cfg.read('./collisionV2/vhs-2d.ini')

vm = SpectralMesh(cfg)
Q_new = FSInelasticVHSCollision(cfg, vm)

In [17]:
%time Q4 = Q_new(bkw_f(Q.v), device='gpu')
%time Q5 = Q.col_sep(bkw_f(Q.v))
np.max(np.abs(Q4 - Q5))

In [33]:
def anisotropic_f(v):
    return 0.8/pi*(np.exp(-(v-2)[:,None]**2/0.25-(v-2)**2/0.25) + np.exp(-(v+0.5)[:,None]**2-(v+0.5)**2))

In [34]:
f0 = anisotropic_f(Q.v)
plot_2d(f0, Q.v, 0)

In [46]:
%time Q1 = Q.col_new(anisotropic_f(Q.v))
%time Q2 = Q.col_sep(anisotropic_f(Q.v))
%time Q3 = Q.col_sep_gpu_copy(anisotropic_f(Q.v))
np.max(np.abs(Q1 - ext_Q(Q.v))), np.max(np.abs(Q2 - ext_Q(Q.v)))
# , np.max(np.abs(Q3 - ext_Q(Q.v)))

In [4]:
%time Q3 = Q_new(anisotropic_f(Q.v), device='gpu')
np.max(np.abs(Q3 - ext_Q(Q.v)))

In [5]:
b = np.arange(3)

In [17]:
a = [None]*3

In [18]:
a[1]

In [12]:
a[0][2] = 2

In [13]:
a

In [6]:
a = np.random.rand(100, 64, 64)
%time Q4 = Q.col_sep(a)
%time Q3 = Q_new(a, device='gpu')
np.max(np.abs(Q3 - Q4))

In [6]:
np.max(np.abs(Q.col_heat(anisotropic_f(Q.v), 0.1) - Q.col_heat_gpu(anisotropic_f(Q.v), 0.1)))

We also have $\rho_0 = 1$, $u_0 = 0$, $E = T$. Temperature $T$ safisfies:

$$
T=\left(T_0-\frac{8\varepsilon}{1-e^2}\right)\exp{\left(-\frac{1-e^2}{4}t\right)}+\frac{8\varepsilon}{1-e^2}.
$$

### Numerical Scheme

In [16]:
a = cp.array([1, 2, 3])

In [25]:
np.newaxis == None

In [24]:
a[(slice(None),) + 3*(np.newaxis,)].device

In [ ]:
from test_2D import TestModule2D
config_2D = get_config('./configs/config_2D.json')

In [11]:
def plot_err(x, err, rate, name):
    fig, ax = plt.subplots()
    ax.loglog(x, err, 'o-', label='numerical error of $T$')
    ax.loglog(x, np.asarray(x)**rate, label='reference: 3-order')
    ax.set_xlabel('$\Delta t$')
    ax.set_ylabel('error of $T$')
    ax.set_title('Convergence of RK3 in $\Delta t$')
    ax.legend(fontsize='large')
    ax.grid(which='both',linestyle=':')
    plt.savefig(name)
    plt.show()

### Time convergence 1

In [3]:
%%time
config_2D.domain_config.S = 3.9
N = [16, 32, 64]
test = TestModule2D(config_2D, bkw_f, N, 0.2, eps=1e-6)

In [4]:
%%time
# Dt = [0.1, 0.05, 0.02, 0.01, 0.005]
# Dt = [0.5, 0.4, 0.2, 0.1, 0.05, 0.04, 0.02, 0.01]
# Dt = [0.1]
Dt = [0.5, 0.1, 0.01]
err, num_f, num_T = test.dt_test(Dt, method='full', N_index=2)

In [ ]:
np.savez('dt_2d_bgk_e_full=0.2', dt=Dt, err=err, num_T=num_T, num_f=num_f)

In [7]:
plot_err(Dt, err, 3, 'dt_2d_bgk_e_full=0.2.pdf')

### Time convergence 2

In [17]:
%%time
config_2D.domain_config.S = 4.0
N = np.array([8, 16, 32, 64, 128])
test = TestModule(config_2D, anisotropic_f, N, 0.2, eps=1e-6)

In [ ]:
%%time
Dt = [0.5, 0.4, 0.2, 0.1, 0.05, 0.04, 0.02, 0.01]
# Dt = [1, 0.8, 0.7, 0.6, 0.5]
err, num_f, num_T = test.dt_test(Dt, method='full')

In [ ]:
np.savez('dt_2d_e_full=0.2', dt=Dt, err=err, num_T=num_T, num_f=num_f)
plot_err(Dt, err, 3, 'dt_2d_e_full=0.2.pdf')

### N convergence 1

In [ ]:
%%time
config_2D.domain_config.S = 3.9

N = np.array([8, 16, 32, 64, 128])
test = TestModule(config_2D, bkw_f, N, 0.2, eps=1e-6)

In [ ]:
%%time
dt = 0.01
err_N, num_f, num_T = test.N_test(dt, method='full')

In [ ]:
np.savez('N_2d_bgk_e_full=0.2', N=N, err=err_N, num_T=num_T, num_f=num_f)

**$\text{tfinal} = 2$, $\Delta t = 0.01$, $e = 0.2$**

| $N$ | Separate | Full |
| ------ | ------ | ------|
| 8 | 9.21116565e-01 | 9.21116565e-01 |
| 16 | 1.27634481e-02 |1.27640374e-02 |
| 32 | 6.79544555e-06 | 6.79745658e-06 |
| 64 | 2.34851361e-10 | 2.36438646e-10 |
| 128 | 6.30565600e-11 | 6.13890050e-11 |

**$\text{tfinal} = 2$, $\Delta t = 0.01$, $e = 0.5$**

| $N$ | Separate | Full |
| ------ | ------ | ------ |
| 8 | 7.98706096e-01 | 7.98706096e-01 |
| 16 | 6.42641236e-03 | 6.42644165e-03 |
| 32 | 4.55713801e-06 | 4.55730861e-06 |
| 64 | 4.93595165e-11 | 4.93770580e-11 |
| 128 | 3.13873372e-11 | 3.14279713e-11 |

**$\text{tfinal} = 2$, $\Delta t = 0.01$, $e = 0.8$**

| $N$ | Separate | Full |
| ------ | ------ | ------|
| 8 | 5.52666966e-01 | 5.52666966e-01 |
| 16 | 4.88821204e-04 | 4.88586534e-04  |
| 32 | 1.14430393e-07 | 1.13897201e-07   |
| 64 | 9.82359749e-11 | 9.82520731e-11   |
| 128 | 1.00099595e-10 | 1.00117470e-10   |

### N convergence 2

In [ ]:
%%time
config_2D.domain_config.S = 4.0

N = np.array([8, 16, 32, 64, 128])

test2 = TestModule(config_2D, anisotropic_f, N, 0.2, eps=1e-6)
test5 = TestModule(config_2D, anisotropic_f, N, 0.5, eps=1e-6)
test8 = TestModule(config_2D, anisotropic_f, N, 0.8, eps=1e-6)

dt = 0.01

err_N2, num_f2, num_T2 = test2.N_test(dt)
err_N5, num_f5, num_T5 = test5.N_test(dt)
err_N8, num_f8, num_T8 = test8.N_test(dt)

err_N2_full, num_f2_full, num_T2_full = test2.N_test(dt, method='full')
err_N5_full, num_f5_full, num_T5_full = test5.N_test(dt, method='full')
err_N8_full, num_f8_full, num_T8_full = test8.N_test(dt, method='full')


np.savez('N_2d_e=0.2', N=N, err=err_N2, num_T=num_T2, num_f=num_f2)
np.savez('N_2d_e=0.5', N=N, err=err_N5, num_T=num_T5, num_f=num_f5)
np.savez('N_2d_e=0.8', N=N, err=err_N8, num_T=num_T8, num_f=num_f8)

np.savez('N_2d_e_full=0.2', N=N, err=err_N2_full, num_T=num_T2_full, num_f=num_f2_full)
np.savez('N_2d_e_full=0.5', N=N, err=err_N5_full, num_T=num_T5_full, num_f=num_f5_full)
np.savez('N_2d_e_full=0.8', N=N, err=err_N8_full, num_T=num_T8_full, num_f=num_f8_full)

**$\text{tfinal} = 2$, $\Delta t = 0.01$, $e = 0.2$**

| $N$ | Separate | Full |
| ------ | ------ | ------|
| 8 | 1.25303916e-01 | 1.25303916e-01 |
| 16 | 1.41601818e-02 |1.42811856e-02 |
| 32 | 1.21162093e-04 | 8.50383206e-05 |
| 64 | 8.65618628e-08 | 5.75217760e-05 |
| 128 | 2.64749862e-08 | 5.74603408e-05 |

**$\text{tfinal} = 2$, $\Delta t = 0.01$, $e = 0.5$**

| $N$ | Separate | Full |
| ------ | ------ | ------ |
| 8 | 9.06935081e-02 | 9.06935081e-02 |
| 16 | 2.06153352e-02 | 2.07345865e-02 |
| 32 | 1.08598123e-04 | 7.68575010e-05 |
| 64 | 3.61540865e-08 | 4.58166915e-05 |
| 128 | 4.84827622e-09 | 4.57852460e-05 |

**$\text{tfinal} = 2$, $\Delta t = 0.01$, $e = 0.8$**

| $N$ | Separate | Full |
| ------ | ------ | ------|
| 8 | 4.21932177e-02 | 4.21932177e-02 |
| 16 | 2.19257970e-02 | 2.17989934e-02  |
| 32 | 1.25546782e-04 | 1.17000137e-04   |
| 64 | 1.55334599e-08 | 2.42867607e-05   |
| 128 |  5.91159477e-09 | 2.42767854e-05   |